In [1]:
import pandas as pd
import matplotlib
import requests
import json
import seaborn as sns
from uszipcode import SearchEngine

#%pip install geocoder
import geocoder

/Users/rubyjoshi/opt/anaconda3/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
pd.set_option('display.max_rows', 300)

In [22]:
vdh_foodPermits = pd.read_csv('VDH_FoodPermits.csv')
main_data = vdh_foodPermits 
main_data

/Users/rubyjoshi/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,HealthDistrict,establishmentName,permitName,permitNumber,program,PermitType,status,RiskCategory,catering,commissary,...,inspectionFrequency,ApplicationDate,OriginalIssueDate,PermitIssueDate,ExpirationDate,FacilityAddress,city,zip,FIPSCode,LocalityName
0,EHD - Rappahannock,NaN,A-La-Carte by Weddings by Ginny,JCRS-5W9Q8L,Food Establishment,Restaurant/Caterer,Surrendered,1,NaN,NaN,...,NaN,2/12/04,6/18/04,6/18/04,6/30/05,NaN,NaN,NaN,NaN,NaN
1,EHD - Prince William,NaN,Chiyoshi,AJOS-ARKQPG,Food Establishment,Carry Out Food Service Only,Business Closed,3,NO,NO,...,NaN,9/22/17,3/29/18,4/1/19,3/31/22,NaN,NaN,NaN,NaN,NaN
2,EHD - Three Rivers,NaN,Monroe Bay Landing,JKAZ-59KLBD,Food Establishment,Restaurant,Surrendered,4,NaN,NaN,...,NaN,1/3/94,2/10/95,2/13/03,2/28/04,NaN,NaN,NaN,NaN,NaN
3,EHD - New River,NaN,Narrows High School-girls Basketball,ACOS-5NUP2H,Food Establishment,Temporary restaurant,Surrendered,NaN,NaN,NaN,...,NaN,6/25/03,6/25/03,7/4/03,7/4/03,NaN,NaN,NaN,NaN,NaN
4,EHD - Virginia Beach,NaN,No. One Chinese Restaurant,JLLD-5CLR4E,Food Establishment,Full Service Restaurant,Surrendered,4,NaN,NaN,...,NaN,7/1/03,7/8/03,7/1/08,7/31/09,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97044,EHD - Alleghany-Roanoke,509 E. Ridgeway Street,The Willow Tree,MCAH-86YFRX,Food Establishment,Fast Food Restaurant,Surrendered,2,NaN,NaN,...,NaN,6/30/10,7/9/10,7/9/10,7/9/11,509 E. Ridgeway Street,Clifton Forge,24422,5.0,CLIFTON FORGE
97045,EHD - Alleghany-Roanoke,117 Falcon Ridge Road,Bowman's Auction Co.,MCAH-85CGVE,Food Establishment,Mobile Food Unit,Surrendered,1,NaN,NaN,...,NaN,4/30/10,5/8/10,5/8/10,5/7/11,117 Falcon Ridge Road,Clifton Forge,24422,5.0,CLIFTON FORGE
97046,EHD - Cumberland Plateau,"Watson, Ray","Watson, Ray",MSTS-9WBLG4,Food Establishment,Other Food Service,Surrendered,NaN,NaN,NaN,...,NaN,8/5/15,NaN,NaN,NaN,431 Craig Street,Norton,24273,195.0,WISE
97047,EHD - Cumberland Plateau,Appalachain Shine II,Appalachain Shine II,KDEL-AMWS5B,Food Establishment,Mobile Food Unit,Expired,NaN,NO,NO,...,NaN,4/20/17,5/23/17,NaN,NaN,120 Ridge Ave,Appalachia,24216,195.0,WISE


### Subset of Status "Permitted" 

In [25]:
# creating a subset clean_vdh that has HealthDistrict and RiskCategory from the dataset

main_data = main_data[main_data['status'] == 'Permitted']

## removing Nan From  LocalityName and also removing out of state 
main_data = main_data.dropna(subset=['LocalityName'])
main_data = main_data.loc[main_data['LocalityName'] != 'OUT OF STATE']
main_data

,HealthDistrict,establishmentName,permitName,permitNumber,program,PermitType,status,RiskCategory,catering,commissary,...,inspectionFrequency,ApplicationDate,OriginalIssueDate,PermitIssueDate,ExpirationDate,FacilityAddress,city,zip,FIPSCode,LocalityName
9553,EHD - Chickahominy,Pasta House Italian Restaurant,Pasta House Italian Restaurant,ELAD-8YFPN2,Food Establishment,Full Service Restaurant,Permitted,3,NO,NO,...,4,9/24/12,9/24/12,10/1/18,9/30/23,8196 Atlee Road,Mechanicsville,23116,85.0,HANOVER
9558,EHD - Chickahominy,Panera Bread Bakery Cafe #6040,Panera Bread Bakery Cafe #6040,085-FD-4659,Food Establishment,Fast Food Restaurant/Caterer,Permitted,2,NO,NO,...,6,2/13/18,11/9/18,1/1/21,12/31/23,810 England St.,Ashland,23005,85.0,HANOVER
9563,EHD - Chickahominy,Paws Cienda Shalom House Retreat Center,Shalom House Retreat,085-FD-5234,Food Establishment,NaN,Permitted,2,NO,YES,...,6,3/22/21,5/21/21,5/21/21,6/30/23,15340 Robert Terrell Rd,Montpelier,23192,85.0,HANOVER
9571,EHD - Chickahominy,Little Bean Coffee Company,Little Bean Coffee Company,085-FD-5105,Food Establishment,NaN,Permitted,1,NO,NO,...,12,8/2/19,10/15/19,2/22/22,12/31/23,7032 Mechanicsville Turnpike,Mechanicsville,23111,85.0,HANOVER
9573,EHD - Chickahominy,El Zocalo Mexican Restaurant,El Zocalo Mexican Restaurant,085-FD-5242,Food Establishment,NaN,Permitted,3,NO,NO,...,4,5/20/21,9/1/21,9/13/22,9/30/23,13271 Hanover Courthouse Rd,Hanover,23069,85.0,HANOVER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97023,EHD - Crater,SSG Pinoy Kitchen,SSG Pinoy Kitchen,670-FD-3436,Food Establishment,NaN,Permitted,2,NO,NO,...,6,9/27/22,10/19/22,11/1/22,10/31/23,224 S 15th Ave,Hopewell,23860,670.0,HOPEWELL
97024,EHD - Crater,Subway #11134,Subway #11134,670-FD-1340,Food Establishment,Fast Food Restaurant,Permitted,2,NO,NO,...,6,6/15/22,9/25/91,8/1/22,7/31/23,601 W Randolph Rd,Hopewell,23860,670.0,HOPEWELL
97036,EHD - Alleghany-Roanoke,"The Way Coffee Shop, LLC",The Way,005-FD-31229,Food Establishment,NaN,Permitted,1,NO,NO,...,12,11/2/20,12/1/20,12/1/22,11/30/23,418 E Ridgeway St,Clifton Forge,24422,5.0,CLIFTON FORGE
97037,EHD - Alleghany-Roanoke,Michael's New York Style Pizza,Michael's New York Style Pizza,#NAME?,Food Establishment,NaN,Permitted,2,YES,NO,...,6,7/1/21,7/1/20,7/1/22,6/30/23,435 E. Ridgeway Street,Clifton Forge,24422,5.0,CLIFTON FORGE


In [26]:
#Filtering permitted data only and converting it to excel

main_data.to_excel("permitted_data.xlsx") 


In [5]:
main_data.columns

Index(['HealthDistrict', 'establishmentName', 'permitName', 'permitNumber',
       'program', 'PermitType', 'status', 'RiskCategory', 'catering',
       'commissary', 'smokingStatus', 'wasteWaterGrease',
       'inspectionFrequency', 'ApplicationDate', 'OriginalIssueDate',
       'PermitIssueDate', 'ExpirationDate', 'FacilityAddress', 'city', 'zip',
       'FIPSCode', 'LocalityName'],
      dtype='object')

## Cleaning the df to remove null counties and null permit types

In [36]:

# Replace vdh_foodPermits with main_data since it's the filtered data ?? 
#Wasn't sure wht  

p_type = vdh_foodPermits['PermitType']
clean_cities = vdh_foodPermits['LocalityName']
newdf = pd.merge(p_type, clean_cities, left_index=True, right_index=True)
print(newdf)


#############
# p_type = main_data['PermitType']
# clean_cities = main_data['LocalityName']
# newdf = pd.merge(p_type, clean_cities, left_index=True, right_index=True)
# print(newdf)


                                  PermitType LocalityName
9553                 Full Service Restaurant      HANOVER
9558            Fast Food Restaurant/Caterer      HANOVER
9580                    Fast Food Restaurant      HANOVER
9584   Public Elementary School Food Service      HANOVER
9586                    Fast Food Restaurant      HANOVER
...                                      ...          ...
96988                   Fast Food Restaurant     HOPEWELL
96993                   Fast Food Restaurant     HOPEWELL
97013  Public Elementary School Food Service     HOPEWELL
97022            Hotel Continental Breakfast     HOPEWELL
97024                   Fast Food Restaurant     HOPEWELL

[20523 rows x 2 columns]


In [7]:
clean_newdf = newdf[newdf['PermitType'].notna()]
merged_df = clean_newdf[clean_newdf['LocalityName'].notna()]
print(merged_df)

                 PermitType   LocalityName
9546       Mobile Food Unit   OUT OF STATE
9547       Mobile Food Unit   OUT OF STATE
9549       Mobile Food Unit   OUT OF STATE
9550       Mobile Food Unit   OUT OF STATE
9551                Caterer   OUT OF STATE
...                     ...            ...
97044  Fast Food Restaurant  CLIFTON FORGE
97045      Mobile Food Unit  CLIFTON FORGE
97046    Other Food Service           WISE
97047      Mobile Food Unit           WISE
97048      Mobile Food Unit           WISE

[76578 rows x 2 columns]


### Removing NaN 

In [30]:
#dropping NaN valuess from Locality column from VDH 

# Here we have zero Nan on PermitType and LocalityName 
main_data  =  main_data.dropna(subset = ['PermitType'])
main_data

,HealthDistrict,establishmentName,permitName,permitNumber,program,PermitType,status,RiskCategory,catering,commissary,...,inspectionFrequency,ApplicationDate,OriginalIssueDate,PermitIssueDate,ExpirationDate,FacilityAddress,city,zip,FIPSCode,LocalityName
9553,EHD - Chickahominy,Pasta House Italian Restaurant,Pasta House Italian Restaurant,ELAD-8YFPN2,Food Establishment,Full Service Restaurant,Permitted,3,NO,NO,...,4,9/24/12,9/24/12,10/1/18,9/30/23,8196 Atlee Road,Mechanicsville,23116,85.0,HANOVER
9558,EHD - Chickahominy,Panera Bread Bakery Cafe #6040,Panera Bread Bakery Cafe #6040,085-FD-4659,Food Establishment,Fast Food Restaurant/Caterer,Permitted,2,NO,NO,...,6,2/13/18,11/9/18,1/1/21,12/31/23,810 England St.,Ashland,23005,85.0,HANOVER
9580,EHD - Chickahominy,Jersey Mikes,Jersey Mikes,085-FD-3567,Food Establishment,Fast Food Restaurant,Permitted,3,NO,NO,...,4,7/25/07,7/25/07,9/22/20,9/30/23,7262 Mechanicsville Turnpike,Mechanicsville,23111,85.0,HANOVER
9584,EHD - Chickahominy,Henry Clay Elementary,Henry Clay Elementary,085-FD-3354,Food Establishment,Public Elementary School Food Service,Permitted,2,NO,NO,...,6,11/7/02,11/21/03,1/1/21,12/31/23,310 South James Street,Ashland,23005,85.0,HANOVER
9586,EHD - Chickahominy,Subway,Subway #20084 - Kings Charter,085-FD-3117,Food Establishment,Fast Food Restaurant,Permitted,2,NO,NO,...,6,11/28/12,11/28/12,1/1/19,12/31/23,9523 Kings Charter Drive #A,Ashland,23005,85.0,HANOVER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96988,EHD - Crater,Hardee’s of Hopewell #2 - #2621,Hardee's of Hopewell #2 - #2621,670-FD-1021,Food Establishment,Fast Food Restaurant,Permitted,2,NO,NO,...,6,6/17/22,10/23/83,8/1/22,7/31/23,310 E Randolph Rd,Hopewell,23860,670.0,HOPEWELL
96993,EHD - Crater,Dunkin/Baskin-Robbins,Dunkin/Baskin-Robbins,670-FD-2098,Food Establishment,Fast Food Restaurant,Permitted,2,NO,NO,...,6,7/14/22,1/10/19,8/1/22,7/31/23,5113 Oaklawn Blvd,Hopewell,23860,670.0,HOPEWELL
97013,EHD - Crater,Dupont Elementary School,Dupont Elementary School,670-FD-2137,Food Establishment,Public Elementary School Food Service,Permitted,2,NO,NO,...,6,12/11/03,1/1/90,1/1/23,12/31/23,300 S 18th Ave,Hopewell,23860,670.0,HOPEWELL
97022,EHD - Crater,Stay Over Suites,Stay Over Suites,670-FD-1250,Food Establishment,Hotel Continental Breakfast,Permitted,1,NO,NO,...,12,1/18/18,1/22/18,8/1/22,7/31/23,4115 Old Woodlawn St,Hopewell,23860,670.0,HOPEWELL


In [31]:
# checking the columns with highest NaN values
missing_num = main_data.isna().sum()
missing_num

HealthDistrict            0
establishmentName         0
permitName                0
permitNumber              0
program                   0
PermitType                0
status                    0
RiskCategory             65
catering               3378
commissary             3378
smokingStatus             0
wasteWaterGrease        208
inspectionFrequency     342
ApplicationDate           0
OriginalIssueDate       220
PermitIssueDate         205
ExpirationDate           39
FacilityAddress           1
city                      4
zip                      37
FIPSCode                  0
LocalityName              0
dtype: int64

## Removing similar permit types 

In [10]:
merged_df = main_data 


In [11]:

# Replace values of 'Full Service Restaurant' with 'Full Service Restaurant/Caterer'
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Full Service Restaurant/Caterer','Full Service Restaurant' )
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Restaurant','Full Service Restaurant' )
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Restaurant/Caterer','Full Service Restaurant' )


# # Replace values of Mobile food unit with  Mobile Food Unit  

merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Mobile food unit', 'Mobile Food Unit') 
##added  Mobile Food Unit PrePackaged 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Mobile Vehicle', 'Mobile Food Unit') 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Mobile Food Unit PrePackaged', 'Mobile Food Unit') 

merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Mobile Unit', 'Mobile Food Unit ') 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Mobile Food Unit', 'Mobile Food Unit')

# # Replace values of Carry out only   with Carry Out Food Service Only 

merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Carry out only' , 'Carry Out Food Service Only')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Carry Out Food Service Only ' , 'Carry Out Food Service Only')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Carry Out' , 'Carry Out Food Service Only')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Carry Out Only' , 'Carry Out Food Service Only')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Carry Out ' , 'Carry Out Food Service Only')

# # Replace values of Temporary restaurant  with Temporary Restaurant 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Temporary restaurant', 'Temporary Restaurant')

# # Replace values of Convenience Store Food Service  with Convenience Store  
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Convenience Store' , 'Convenience Store Food Service')

# # Replace values of Child Care Center with Child Care Food Service
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Child Care Center', 'Child Care Food Service')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Child care', 'Child Care Food Service')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Child Care', 'Child Care Food Service')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Child Care With Food ', 'Child Care Food Service')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Residential Child Care Institution Food Service', 'Child Care Food Service')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Child Care Kitchen', 'Child Care Food Service') 

# # Replace values of Summer Food Service Program Kitchen  with Summer Food Service Program Feeding Site 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Summer Food Service Program Kitchen', 'Summer Food Service Program Feeding Site')

# # Replace Nursing homes with Nursing Home Food Service
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Nursing homes', 'Nursing Home Food Service')

# # Replace Seasonal Full Service Restaurant with Seasonal restaurant
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Seasonal Full Service Restaurant' , 'Seasonal restaurant ')

# # Replace Seasonal restaurant  with Seasonal restaurant ( Gaps )  Seasonal Kitchen , Seasonal Restaurant
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Seasonal restaurant ', 'Seasonal Restaurant') 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Seasonal', 'Seasonal Restaurant') 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Seasonal Kitchen', 'Seasonal Restaurant') 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Seasonal restaurant', 'Seasonal Restaurant') 


# # # Replace Nursing homes with Adult Care Home Food Service  Adult Care/Hospital 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Adult Care Home', 'Adult Care Home Food Service')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Adult Home', 'Adult Care Home Food Service')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Adult day care', 'Adult Care Home Food Service')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Adult care homes', 'Adult Care Home Food Service')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Adult Day Care Food Service', 'Adult Care Home Food Service')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Adult Care/Hospital', 'Adult Care Home Food Service')
# Replace Group Home', 'Group Home Food Service'
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Group Home', 'Group Home Food Service')

# Other Service 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Other food service', 'Other Food Service')

# Jail Kitchen 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Jail kitchen', 'Jail Food Service')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Jail Food Service', 'Jail Food Service')


# # Replace Seasonal restaurant  with Fast Food Restaurant 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Fast Food Restaurant/Caterer', 'Fast Food Restaurant') 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Fast Food', 'Fast Food Restaurant') 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Fast Food Restaurant', 'Fast Food Restaurant') 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Seasonal Fast Food Restaurant', 'Fast Food Restaurant') 

# Private school kitchen  with Private School Kitchen
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Private school kitchen', 'Private School Kitchen') 





# Convinience store   Convenience store , 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Convenience store', 'Convenience Store Food Service')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Local Convenience Store', 'Convenience Store Food Service')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Convenience Store (Local)', 'Convenience Store Food Service')
##added
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Convenience Store', 'Convenience Store Food Service')

# Private college food service
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Private college food service', 'Private College Food Service')


#summer camp 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Summer camp kitchen', 'Summer Camp Food Service')




# # Bed and Brekfast food service 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Bed & Breakfast', 'Bed & Breakfast Food Service') 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Bed & Breakfast food service', 'Bed & Breakfast Food Service')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Continental Breakfast', 'Bed & Breakfast Food Service')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Hotel Continental Breakfast', 'Bed & Breakfast Food Service')

#Hospital Kitchcen 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Hospital kitchen', 'Hospital Food Service')


# Replace Seasonal restaurant  with Fast Food Restaurant 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Vending Machines', 'Vending Machine')

# Replace Grocery stores
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Grocery Store (Bakery Dept.)', 'Grocery Store')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Grocery Store (Grocery Dept.)', 'Grocery Store')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Grocery Store (Meat/Poultry Dept.)', 'Grocery Store')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Grocery Store Food Service', 'Grocery Store')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Local Grocery Store Food Service', 'Grocery Store') 
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Local Grocery Store', 'Grocery Store')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Grocery store', 'Grocery Store')
merged_df.loc['PermitType'] = merged_df['PermitType'].replace('Small Grocery Store ', 'Grocery Store')


/Users/rubyjoshi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
/Users/rubyjoshi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1700: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, v, pi)


In [12]:
merged_df['PermitType'].value_counts()
# merged_df['PermitType'].unique()

Full Service Restaurant                            6915
Fast Food Restaurant                               5476
Child Care Food Service                            1128
Public Elementary School Food Service              1112
Mobile Food Unit                                    779
Carry Out Food Service Only                         699
Public Middle or High School Food Service           696
Hotel Continental Breakfast                         466
Other Food Service                                  382
Seasonal Fast Food Restaurant                       373
Adult Care Home Food Service                        333
Caterer                                             274
Full Service Restaurant/Caterer                     246
Nursing Home Food Service                           227
Fast Food Restaurant/Caterer                        172
State College Food Service                          136
Convenience Store Food Service                      128
Private Elementary School Food Service          

In [13]:

print(merged_df)
type(merged_df)

                      HealthDistrict                        establishmentName  \
9547                   EHD - Fairfax             Capital Chicken & Waffles #1   
9552        EHD - Cumberland Plateau             Mooney's Wheelin' & Grillin'   
9553              EHD - Chickahominy           Pasta House Italian Restaurant   
9558              EHD - Chickahominy           Panera Bread Bakery Cafe #6040   
9563              EHD - Chickahominy  Paws Cienda Shalom House Retreat Center   
...                              ...                                      ...   
97024                   EHD - Crater                            Subway #11134   
97036        EHD - Alleghany-Roanoke                 The Way Coffee Shop, LLC   
97037        EHD - Alleghany-Roanoke           Michael's New York Style Pizza   
97043        EHD - Alleghany-Roanoke                   509 E. Ridgeway Street   
PermitType                       NaN                                      NaN   

                           

pandas.core.frame.DataFrame

## Total permit types

In [14]:
totalDf = merged_df.LocalityName.value_counts().reset_index(name='PermitType') #.iloc[:,1:2]
totalDf

,index,PermitType
0,FAIRFAX COUNTY,3356
1,VIRGINIA BEACH,1758
2,LOUDOUN,1329
3,HENRICO,1111
4,NORFOLK,1082
5,CHESTERFIELD,973
6,PRINCE WILLIAM,935
7,ARLINGTON,906
8,CHESAPEAKE,694
9,NEWPORT NEWS,676


In [15]:
pd.set_option('display.max_rows', 200)

In [16]:
display(merged_df)

,HealthDistrict,establishmentName,permitName,permitNumber,program,PermitType,status,RiskCategory,catering,commissary,...,inspectionFrequency,ApplicationDate,OriginalIssueDate,PermitIssueDate,ExpirationDate,FacilityAddress,city,zip,FIPSCode,LocalityName
9547,EHD - Fairfax,Capital Chicken & Waffles #1,Capital Chicken & Waffles #1,AGAA-BAHKGU,Food Establishment,Mobile Food Unit,Permitted,1,NaN,NaN,...,12,3/21/19,3/21/19,3/21/19,12/31/19,"938 E. Swann Creek Rd, # 718",FORT WASHINGTON,20744,124.0,OUT OF STATE
9552,EHD - Cumberland Plateau,Mooney's Wheelin' & Grillin',Mooney's Wheelin' & Grillin',124-FD-2204,Food Establishment,NaN,Permitted,3,NO,YES,...,4,6/28/22,7/7/22,7/7/22,7/31/23,225 Larchwood Road,Bluefield,24701,124.0,OUT OF STATE
9553,EHD - Chickahominy,Pasta House Italian Restaurant,Pasta House Italian Restaurant,ELAD-8YFPN2,Food Establishment,Full Service Restaurant,Permitted,3,NO,NO,...,4,9/24/12,9/24/12,10/1/18,9/30/23,8196 Atlee Road,Mechanicsville,23116,85.0,HANOVER
9558,EHD - Chickahominy,Panera Bread Bakery Cafe #6040,Panera Bread Bakery Cafe #6040,085-FD-4659,Food Establishment,Fast Food Restaurant/Caterer,Permitted,2,NO,NO,...,6,2/13/18,11/9/18,1/1/21,12/31/23,810 England St.,Ashland,23005,85.0,HANOVER
9563,EHD - Chickahominy,Paws Cienda Shalom House Retreat Center,Shalom House Retreat,085-FD-5234,Food Establishment,NaN,Permitted,2,NO,YES,...,6,3/22/21,5/21/21,5/21/21,6/30/23,15340 Robert Terrell Rd,Montpelier,23192,85.0,HANOVER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97024,EHD - Crater,Subway #11134,Subway #11134,670-FD-1340,Food Establishment,Fast Food Restaurant,Permitted,2,NO,NO,...,6,6/15/22,9/25/91,8/1/22,7/31/23,601 W Randolph Rd,Hopewell,23860,670.0,HOPEWELL
97036,EHD - Alleghany-Roanoke,"The Way Coffee Shop, LLC",The Way,005-FD-31229,Food Establishment,NaN,Permitted,1,NO,NO,...,12,11/2/20,12/1/20,12/1/22,11/30/23,418 E Ridgeway St,Clifton Forge,24422,5.0,CLIFTON FORGE
97037,EHD - Alleghany-Roanoke,Michael's New York Style Pizza,Michael's New York Style Pizza,#NAME?,Food Establishment,NaN,Permitted,2,YES,NO,...,6,7/1/21,7/1/20,7/1/22,6/30/23,435 E. Ridgeway Street,Clifton Forge,24422,5.0,CLIFTON FORGE
97043,EHD - Alleghany-Roanoke,509 E. Ridgeway Street,Museum of Antique Coffee Mills,005-FD-31126,Food Establishment,NaN,Permitted,1,YES,NO,...,12,11/22/19,12/1/19,1/1/22,12/31/22,509 E. Ridgeway Street,Clifton Forge,24422,5.0,CLIFTON FORGE


## Group by the counties and permit types

In [17]:
result2 = merged_df.groupby(['LocalityName', 'PermitType']).size().reset_index(name='Count')
(result2)


,LocalityName,PermitType,Count
0,ACCOMACK,Adult Day Care Food Service,1
1,ACCOMACK,Carry Out Food Service Only,1
2,ACCOMACK,Caterer,1
3,ACCOMACK,Child Care Food Service,4
4,ACCOMACK,Convenience Store Food Service,4
...,...,...,...
2007,YORK,Nursing Home Food Service,2
2008,YORK,Private Elementary School Food Service,1
2009,YORK,Public Elementary School Food Service,10
2010,YORK,Public Middle or High School Food Service,7


In [18]:
result2.to_excel("output2.xlsx") 

In [19]:
print(result2["PermitType"].unique())

['Adult Day Care Food Service' 'Carry Out Food Service Only' 'Caterer'
 'Child Care Food Service' 'Convenience Store Food Service'
 'Fast Food Restaurant' 'Fast Food Restaurant/Caterer'
 'Full Service Restaurant' 'Full Service Restaurant/Caterer'
 'Hospital Food Service' 'Hotel Continental Breakfast' 'Mobile Food Unit'
 'Nursing Home Food Service' 'Other Food Service'
 'Private College Food Service' 'Public Elementary School Food Service'
 'Public Middle or High School Food Service'
 'Seasonal Full Service Restaurant' 'Adult Care Home Food Service'
 'Commissary' 'Dept. of Juvenile Justice Food Service'
 'Grocery Store Food Service' 'Jail Food Service'
 'Private Elementary School Food Service'
 'Private Middle or High School Food Service'
 'Seasonal Fast Food Restaurant' 'State College Food Service'
 'Summer Camp' 'Summer Camp Food Service' 'Group Home Food Service'
 'State Institution Food Service' 'Summer Food Service Program Kitchen'
 'Bed & Breakfast Food Service' 'Building'
 'Local